<a href="https://colab.research.google.com/github/carolvieirav/MaeTechDataRepo/blob/main/Sistema_de_Recomenda%C3%A7%C3%A3o_de_Usu%C3%A1rias_M%C3%A3eTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'rake_nltk'

In [2]:
import pandas as pd
import numpy as np
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

### **Creating DataFrames with Job Openings and Users informations**

In [24]:
# Real Job Openings descripions posted on linkedin by ProviHack Woman partner companies

companies=[['Pagar.me', 'Data Developer | Analytics Engineer',True,
           'Plano de Saúde e Odontológico Bradesco (sem coparticipação),Vale transporte, vale Refeição e/ou Vale Alimentação, Seguro de Vida, Gympass,Auxílio Creche',
           ' NaN',' NaN', 'SQL, airflow, luigi, AWS, snowFlake, Big Query ou Azure SQL DW', 'NaN', 'NaN','Home Office']
]

In [25]:
users = [['Ana', 'Ciêntista de Dados', True, 'Vale Alimentação', 4000, 'NaN', 'SQL, Python, Tableau, Power BI, AWS', 
             'NaN', 'NaN', 'Pós Graduação', 'Home Office']
            ]

In [26]:
COMP_COL = [
    'Nome da Empresa',
    'Cargo',
    'Flexibilidade',
    'Benefícios',
    'Salário',
    'PJ ou CLT',
    'Tecnologias',
    'Tempo de experiência'
    'Area de experiência',
    'Escolaridade',
    'Escritório ou HomeOffice'
]

compdf = pd.DataFrame(companies, columns=COMP_COL)
compdf

,Nome da Empresa,Cargo,Flexibilidade,Benefícios,Salário,PJ ou CLT,Tecnologias,Tempo de experiênciaArea de experiência,Escolaridade,Escritório ou HomeOffice
0,Pagar.me,Data Developer | Analytics Engineer,True,Plano de Saúde e Odontológico Bradesco (sem co...,NaN,NaN,"SQL, airflow, luigi, AWS, snowFlake, Big Query...",NaN,NaN,Home Office


In [28]:
USERS_COL = [
    'Nome Completo',
    'Cargo',
    'Flexibilidade',
    'Benefícios',
    'Salário',
    'PJ ou CLT',
    'Tecnologias',
    'Tempo de experiência',
    'Area de experiência',
    'Escolaridade',
    'Escritório ou HomeOffice'
]

usersdf = pd.DataFrame(users, columns=USERS_COL)
usersdf

,Nome Completo,Cargo,Flexibilidade,Benefícios,Salário,PJ ou CLT,Tecnologias,Tempo de experiência,Area de experiência,Escolaridade,Escritório ou HomeOffice
0,Ana,Ciêntista de Dados,True,Vale Alimentação,4000,NaN,"SQL, Python, Tableau, Power BI, AWS",NaN,NaN,Pós Graduação,Home Office


### **Data Cleaning**

Extracting the key words from a text with a feature from the nltk package

In [29]:
#I chose to clean only the column of technologies for now to optimize time

for index, row in compdf.iterrows():
  for index, row in usersdf.iterrows():
    tecnologias = row['Tecnologias']
   
    # instantiating Rake
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(tecnologias)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the column
    row['Tecnologias'] = list(key_words_dict_scores.keys())

### **Modeling**

In [30]:
# instantiating and generating the count matrix
count = CountVectorizer()
comp_matrix = count.fit_transform(compdf['Tecnologias'])
user_matrix = count.fit_transform(usersdf['Tecnologias'])

# generating the cosine similarity matrix
comp_sim = cosine_similarity(comp_matrix, comp_matrix)
user_sim = cosine_similarity(user_matrix, user_matrix)
opening_sim = cosine_similarity(comp_sim, user_sim)

In [31]:
# creating a Series for the job openings so they are associated to an ordered numerical list
indices = pd.Series(compdf.index)

#  defining the function that takes in a companny post as input and returns the top recommended users
def recommendations(indices, opening_sim = opening_sim):
    # initializing the empty list of recommended users
    recommended_users = []

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(opening_sim[indices]).sort_values(ascending = False)

    # getting the indexes of the most similar users
    top_indexes = list(score_series.iloc[0:4].index)
    
    # populating the list with the recommendations
    for i in top_indexes:
        recommended_users.append(list(compdf.index)[i])
        
    return recommended_users

In [32]:
recommendations(0)

[0]